In [1]:
#packages
import numpy
import pandas as pd
import os
from riotwatcher import LolWatcher, ApiError

In [2]:
#need to put this into a class
lol_watcher = LolWatcher('RGAPI-c9a0d6bf-69ef-410a-94e6-fda7c2bc19a3') #api key

In [187]:
#####################
## Players 
#####################

def get_Player(region, name):
    
    valid_region_names = ['oc1', 'na1', 'eun1']
    if region in valid_region_names:
        player = lol_watcher.summoner.by_name(region, name.lower())
        return lol_watcher.league.by_summoner('oc1', player['id'])
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')    

#####################
## Patches 
#####################
        
def get_CurrentPatchVersions(region):
    
    valid_region_names = ['oce', 'na1', 'eune']
    if region in valid_region_names:
        return lol_watcher.data_dragon.versions_for_region(region)
    else:
        raise ValueError(f'region name is not a known region. Try one of {valid_region_names}')   

def get_AllPatchVersions():
    
    return [i for i in lol_watcher.data_dragon.versions_all() if not i.startswith('lolpatch')]

#####################
## Champions
#####################

def get_Champions(patchversion = 'latest'):
    
    if patchversion == 'latest':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in [get_AllPatchVersions()[0]]] #nested by patch        
    elif patchversion == 'all':
        champions = [[(patch,champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in get_AllPatchVersions()] #nested by patch
    else:
        champions = [[(patch, champ) for champ in lol_watcher.data_dragon.champions(patch)['data']] for patch in patchversion] #nested by patch
        
    return pd.DataFrame([val for sublist in champions for val in sublist], columns = ['patchversion', 'champion']) #compress into single list

def get_ChampionStats(champions):
        data = pd.DataFrame.from_dict([{
                'patch': patchversion, 
                'champion': champion, 
                'championtype': lol_watcher.data_dragon.champions(patchversion)['data'][champion]['tags'],
                'spellresource': lol_watcher.data_dragon.champions(patchversion)['data'][champion]['partype'],
                **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['info'],
                **lol_watcher.data_dragon.champions(patchversion)['data'][champion]['stats']} 
            for patchversion, champion in champions.values.tolist()
        ])
        
        #create championtype
        return pd.concat([data, pd.get_dummies(data['championtype'].apply(pd.Series).stack(), prefix = 'championtype').sum(level=0)], axis = 1).drop(columns=['championtype'])
    

#####################
## Items
#####################    
    
#lol_watcher.data_dragon.items('12.22.1')

In [188]:
champions = get_Champions()[0:3]
champions

,patchversion,champion
0,12.22.1,Aatrox
1,12.22.1,Ahri
2,12.22.1,Akali


In [189]:
data = get_ChampionStats(champions)
data

,patch,champion,spellresource,attack,defense,magic,difficulty,hp,hpperlevel,mp,...,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed,championtype_Assassin,championtype_Fighter,championtype_Mage,championtype_Tank
0,12.22.1,Aatrox,Blood Well,8,4,3,4,650,114,0,...,0,0,60,5.0,2.5,0.651,0,1,0,1
1,12.22.1,Ahri,Mana,3,4,8,5,570,96,418,...,0,0,53,3.0,2.0,0.668,1,0,1,0
2,12.22.1,Akali,Energy,5,3,8,7,570,119,200,...,0,0,62,3.3,3.2,0.625,1,0,0,0


# Data